In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
import os
from string import ascii_lowercase
import csv
from tqdm import tqdm
import pandas as pd

In [22]:
os.getcwd()
home_dir = r'c:\\Users\\tutha\\Desktop\\Hacked2023\\ASL_Translation-NAIT\\'
ds_dir = r'c:\\Users\\tutha\\Desktop\\Hacked2023\\ASL_Translation-NAIT\\dataset\\'
newcoming_ds_dir = r'c:\\Users\\tutha\\Desktop\\Hacked2023\\ASL_Translation-NAIT\\letters\\'
old_ds_dir = r'c:\\Users\\tutha\\Desktop\\Hacked2023\\ASL_Translation-NAIT\\old_dataset\\'

for c in ascii_lowercase:
    
    combined_csv = pd.concat([pd.read_csv(old_ds_dir + c + ".csv", header = None), pd.read_csv(newcoming_ds_dir + c + ".csv", header = None)], axis=0, ignore_index=True)
    combined_csv.to_csv(ds_dir + c +".csv", index=False, header = False)

In [23]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.data = {"data":[], "label":[]}
        self.length = 0
        class_index = 0
        for c in ascii_lowercase:
            with open(ds_dir + c + ".csv") as csv_file:
                reader = csv.reader(csv_file)
                for row in reader:
                    int_row = [int(element) for element in row]
                    self.data["data"].append(torch.tensor(int_row, dtype=torch.float32))
                    self.data["label"].append(torch.tensor(class_index))
                    self.length += 1
            class_index += 1

        for i in range(len(self.data["data"])):
            self.data["data"][i] /=  4095
    def __getitem__(self, index):
        return self.data["data"][index].unsqueeze(0), self.data["label"][index]
    def __len__(self):
        return self.length

In [24]:
dataset = CustomDataset()
len(dataset)

44071

In [43]:
trainset, testset = torch.utils.data.random_split(dataset, [39664,4407])
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

trainset[0], trainset[1]


((tensor([[1.0000, 0.3746, 0.7140, 0.8747, 0.1741, 0.7187, 0.9810]]),
  tensor(25)),
 (tensor([[0.9917, 0.1338, 0.6962, 0.9199, 0.2899, 0.8899, 1.0000]]),
  tensor(13)))

In [32]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_layers = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3),
                                    nn.Dropout1d(0.25),
                                    nn.ReLU(),
                                    nn.Conv1d(in_channels=4, out_channels=8, kernel_size=2),
                                    nn.Dropout1d(0.25),
                                    nn.ReLU(),
                                    nn.Conv1d(in_channels=8, out_channels=16, kernel_size=2),
                                    nn.Dropout1d(0.25),
                                    nn.ReLU(),
                                    )
        
        self.linear = nn.Linear(16 * 3, 64)
        self.dropout = nn.Dropout(0.25)
        self.out = nn.Linear(64, 26)
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = nn.Flatten()(x)
        x = nn.ReLU()(self.dropout(self.linear(x)))
        x = self.out(x)
        #x = nn.LogSoftmax()(x)
        return x
    
test = Model()
torchinfo.summary(test, (8, 1, 7))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [8, 26]                   --
├─Sequential: 1-1                        [8, 16, 3]                --
│    └─Conv1d: 2-1                       [8, 4, 5]                 16
│    └─Dropout1d: 2-2                    [8, 4, 5]                 --
│    └─ReLU: 2-3                         [8, 4, 5]                 --
│    └─Conv1d: 2-4                       [8, 8, 4]                 72
│    └─Dropout1d: 2-5                    [8, 8, 4]                 --
│    └─ReLU: 2-6                         [8, 8, 4]                 --
│    └─Conv1d: 2-7                       [8, 16, 3]                272
│    └─Dropout1d: 2-8                    [8, 16, 3]                --
│    └─ReLU: 2-9                         [8, 16, 3]                --
├─Linear: 1-2                            [8, 64]                   3,136
├─Dropout: 1-3                           [8, 64]                   --
├─Linear: 1

In [37]:
from torch.utils.tensorboard import SummaryWriter

def train(train_loader, model, epochs, loss_function):
    writer = SummaryWriter()
    model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-05)
    
    with tqdm(total=epochs) as pbar:
        for epoch in range(epochs):
            model.train()
            model_loss = 0.0
            
            for batch_idx, (data,target) in enumerate(train_loader):
                data = data.cuda().type(torch.cuda.FloatTensor)
                               
                onehot_target = F.one_hot(target, num_classes=26).cuda().type(torch.cuda.FloatTensor)
                
                optimizer.zero_grad()
                
                prediction = model(data)
                
                loss = loss_function(prediction, onehot_target)
                    
                loss.backward()
                optimizer.step()
                model_loss += loss/len(train_loader)
                
            pbar.update(1)
            pbar.set_postfix(**{'train__loss': model_loss.item()})   
            writer.add_scalar("Train loss", model_loss.item(), epoch)
            
            
    
    torch.save(model, "ASL_interpreter.pt")
    writer.close()

In [39]:
model = Model()
epochs = 50
loss_function = nn.CrossEntropyLoss()

train(train_loader, model, epochs, loss_function)

100%|██████████| 50/50 [14:57<00:00, 17.94s/it, train__loss=0.785]


In [44]:
def test(test_loader, model_name):
    
    file_to_load = model_name + ".pt"
    test_model = Model()
    test_model = torch.load(file_to_load).cuda()
    
    prediction_loss = nn.CrossEntropyLoss()
    
    test_model.eval()
    correct = 0
    loss = 0
    for batch_idx, (data,target) in enumerate(test_loader):
        
        data = data.cuda().type(torch.cuda.FloatTensor)
        onehot_target = F.one_hot(target, num_classes=26).cuda().type(torch.cuda.FloatTensor)
        prediction = test_model(data).cuda()
        
        
        loss += prediction_loss(prediction,onehot_target)
        pred_class = prediction.data.max(1, keepdim=True)[1]
        correct += pred_class.eq(target.cuda().data.view_as(pred_class)).sum()

    print(loss.item()/len(testset))
    print(correct.item()/len(testset))
    

test(test_loader, "ASL_interpreter")
        
    

0.4171623159001305
0.8706603131381893
